In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns


import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Analisis de Data

In [ ]:
airline = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/train.csv")
airline

In [ ]:
print(airline.isnull().sum())

In [ ]:
airline = airline.dropna()
sns.heatmap(airline.isnull(), cbar=False);

In [ ]:
fig = px.scatter(
    x=airline["Customer Type"], 
    y=airline["Flight Distance"], 
    color=airline["satisfaction"],
)
fig.show()

In [ ]:
fig = px.scatter(
    x=airline["Age"], 
    y=airline["Flight Distance"], 
    color=airline["satisfaction"],
)
fig.show()

In [ ]:
airline.info()

In [ ]:
airline["satisfaction"].unique()

In [ ]:
airline["Gender"] = (airline["Gender"].astype('category')).cat.codes # 1 Male / 0 Female 
airline["Customer Type"] = (airline["Customer Type"].astype('category')).cat.codes #0 Loyal / 1 Disloyal
airline["Type of Travel"] = (airline["Type of Travel"].astype('category')).cat.codes # 1 Personal / 0 Business
airline["Class"] = (airline["Class"].astype('category')).cat.codes # 2 Eco Plus / 0 Business / 1 Eco
airline["satisfaction"] = (airline["satisfaction"].astype('category')).cat.codes #0 neutral or dissatisfied / 1 satisfied
airline

In [ ]:
#parallel_coordinates(airline,'satisfaction', color=('#556270', '#4ECDC4', '#C7F464', '#e69509','#f72f8c','#d681f7'))
#plt.show()

In [ ]:
corr = airline.corr()
corr.style.background_gradient(cmap='plasma').set_precision(2)

# Split Data

In [ ]:
y = airline['satisfaction']
airline = airline.drop(['satisfaction', 'Unnamed: 0', 'id'], axis=1)
X_train, X_test, Y_train, Y_test= train_test_split(airline, y, test_size=0.2, random_state=24)

# Ajuste de Modelo


In [ ]:
model = GaussianNB()
model.fit(X_train, Y_train)

In [ ]:
accuracy = model.score(X_train, Y_train)
test_accuracy = model.score(X_test, Y_test)
print(f"accuracy: {accuracy}, test_accuracy: {test_accuracy}")

# Matriz de confusion 

In [ ]:
predictions = model.predict(X_train)
cm = confusion_matrix(Y_train, predictions)
fig = px.imshow(cm, x=['neutral or dissatisfied', 'satisfied'], y=['neutral or dissatisfied', 'satisfied'])
fig.show()

In [ ]:
predictions = model.predict(X_test)
cm = confusion_matrix(Y_test, predictions)
fig = px.imshow(cm, x=['neutral or dissatisfied', 'satisfied'], y=['neutral or dissatisfied', 'satisfied'])
fig.show()

# Contornos y LDA

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Contour(
    x=airline['Flight Distance'],
    y=airline['Age'],
    z=model.predict(X_test),
    showscale=False,
    opacity=0.40,
    colorscale='sunset'
), row=1, col=1)

fig.add_trace(go.Scatter(
     x=airline['Flight Distance'],
    y=airline['Age'],
    text=Y_test,
    mode='markers',
    marker_symbol=Y_test,
    marker=dict(color=y, colorscale='sunset')
), row=1, col=1)

fig.update_layout(showlegend=False)

fig.show()

In [ ]:
airline

In [ ]:
lda_model = LinearDiscriminantAnalysis(n_components=1).fit(airline, y)

reduced_X = lda_model.transform(airline).reshape(-1, 1)

reduced_X 

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Contour(
    x=reduced_X.ravel(), 
    y=np.linspace(-10, 10, 150),
    z=model.predict(reduced_X),
    showscale=False,
    opacity=0.40,
    colorscale='portland'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=reduced_X.ravel(), 
    y=[0 for x in range(len(reduced_X))],
    text=y,
    mode='markers',
    marker_symbol=y,
    marker=dict(color=y, colorscale='portland')
), row=1, col=1)

fig.update_layout(yaxis_range=[-0.01,0.01])

fig.show()